In [1]:
# Filename: methods.py
# Purpose: Contains the logic behind getting predictions based on user input.

# Import libraries:
# import plotly.express as px
import pickle
from datetime import date, datetime

from fbprophet.plot import plot_plotly
import plotly.offline as py


In [2]:
# Load the model:
def load_model(country):
    path = 'static/model/' + str(country) + '.pckl'
    with open(path, 'rb') as fin:
        model = pickle.load(fin)

    return model


# Convert the date into count of days from today's date:
def count_days(f_date):
    current_date = date.today()
    future_date = datetime.strptime(f_date, '%Y-%m-%d')
    delta = future_date.date() - current_date

    return delta.days


# Predict the number of cases based on user selected input (chosen country + chosen future date):
def predict(country, f_date):
    model = load_model(country)
    days = count_days(f_date)+2

    future = model.make_future_dataframe(periods=days)
    forecast = model.predict(future)

    predicted_object = forecast[['ds', 'yhat']].iloc[-1]
    today_object = forecast[['ds', 'yhat']].iloc[-days]

    fig = plot_plotly(model, forecast, xlabel='Date', ylabel='Confirmed Cases')
    fig.update_layout(
        autosize=False,
        width=800,
        height=500
    )
    py.plot(fig, filename='static/graph/graph.html', auto_open=False)

    prediction = {
        "today_date": today_object['ds'].date(),
        "today_cases": int(round(today_object['yhat'])),
        "predict_date": predicted_object['ds'].date(),
        "predict_cases": int(round(predicted_object['yhat']))
    }

    return prediction